In [8]:
# Execute if necessary
# %%capture
# !pip install numpy seaborn matplotlib pandas openml

In [9]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, Tuple, Union, List
import openml
import sklearn

# Práctica 5: Introducción a Modelos de Clasificación

__Instrucciones__: A continuación hay una lista de funciones que debe implementar o tareas que debe desarrollar. La descripción de cada una de ellas se encuentra en la definición de cada una de las funciones.

La entrega de la práctica será en la siguiente sesión a menos que la indicación sea otra. La revisión iniciará al iniciar la sesión y únicamente podrá ser evaluada durante la duración de la sesión.

## Asignación 1

Implemente una clase  que permita ajustar un modelo KNN. La clase debe cumplir con los siguientes requisitos:

- Debe contar con un método para realizar el entrenamiento en caso de ser requerido.
- Debe contar con un método para realizar las predicciones.
- Cada método debe imprimir una barra de progreso que permita conocer el tiempo estimado en que terminará.
- La clase debe permitir el uso de la distancia Mikownski y similitud coseno.

El algoritmo k-vecinos más cercanos (KNN) es un simple algoritmo de aprendizaje automático supervisado que se puede utilizar para resolver problemas de clasificación y regresión. Es fácil de implementar y comprender, pero tiene un inconveniente importante de volverse significativamente más lento a medida que crece el tamaño de los datos en uso.

KNN trabaja buscando las distancias entre una consulta y todos los ejemplos en los datos, seleccionando el número especificado ejemplos (K) más cercanos a la consulta, luego vota por la etiqueta más frecuente (en el caso de la clasificación) o promedia las etiquetas (en el caso de la regresión).

En el caso de la clasificación y la regresión, vimos esa elección la K correcta para nuestros datos se hace probando varios Ks y escogiendo el que mejor funciona.

In [10]:
import time
import numpy as np
from tqdm import tqdm

class KNNModel:

    def __init__(self, k=5, metric='euclidean' , p=2):
        self.k = k
        self.metric = metric
        self.p = p

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        start_time = time.time()
        predictions = []
        for x_test in tqdm(X):
            distances = []
            for x_train in self.X_train:

                #identificamos el metodo de distancia a utilizar

                if self.metric == 'euclidean':
                    distance = np.sqrt(np.sum((x_test - x_train)**2))
                elif self.metric == 'cosine':
                    distance = 1 - (np.dot(x_test, x_train) / (np.linalg.norm(x_test) * np.linalg.norm(x_train)))
                elif self.metric == 'manhattan':
                    distance = np.sum(np.abs(x_test - x_train))
                elif self.metric == 'mikownski':
                    
                    distance = (np.sum((x_test - x_train)**self.p))**(1/self.p)
                else:
                    raise ValueError('Metrica no soportada')

                distances.append(distance)
            #evaluamos la distancia de cada punto con el resto de los puntos
            top_k_indices = np.argsort(distances)[:self.k]
            top_k_classes = [self.y_train[i] for i in top_k_indices]
            most_common_class = max(set(top_k_classes), key=top_k_classes.count)
            predictions.append(most_common_class)
        
        elapsed_time = time.time() - start_time
        print(f"Prediccion completada en {elapsed_time:.2f} segundos.")
        return np.array(predictions)

    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions == y)


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer

# Cargar datos de ejemplo
data = load_breast_cancer()
X, y = data.data, data.target

# Normalizar los datos
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar modelo KNN
model = KNNModel(k=5, metric='mikownski' , p=2)
model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular la puntuación en el conjunto de prueba
score = model.score(X_test, y_test)

# Imprimir precisión
print(f"Precisión: {accuracy_score(y_test, y_pred):.4f}")


100%|██████████| 114/114 [00:00<00:00, 601.21it/s]


Prediccion completada en 0.20 segundos.


100%|██████████| 114/114 [00:00<00:00, 609.54it/s]

Prediccion completada en 0.19 segundos.
Precisión: 0.9474


## Asignación 2

Implemente una clase que permita ajustar un modelo Naive Bayes. La clase debe cumplir con los siguientes requisitos:

- Debe contar con un método para realizar el entrenamiento en caso de ser requerido.
- Debe contar con un método para realizar las predicciones.
- Cada método debe imprimir una barra de progreso que permita conocer el tiempo estimado en que terminará.

En esta sección se incluye un dataset real. El dataset importado se llama _spambase_ y puede leer su descripción en la siguiente liga

https://www.openml.org/d/42904

In [12]:
# Descarga la metadata del dataset
dataset_info = openml.datasets.get_dataset(42904, download_data=False)

target = "CLASS"

(
    features, # Dataframe con las características que se pueden utilizar para predecir
    outputs, # Columna a predecir
    _, # Máscara que indica que columnas de todas las características son categoricas
    columns # Lista con el nombre de las características
)= dataset_info.get_data(
    dataset_format="dataframe", target=target
)

columns = np.array(columns)

In [13]:
print(f"La columna a predecir se llama '{target}'")
print(f"Todas las características son {str(columns)}")

La columna a predecir se llama 'CLASS'
Todas las características son ['COMMENT_ID' 'AUTHOR' 'DATE' 'CONTENT']


In [14]:
# Impresión de las características
features
# Eliminar las características que no se pueden utilizar para predecir
features = features.drop(columns=["COMMENT_ID", "AUTHOR", "DATE"])

In [22]:
features

,CONTENT
0,"Huh, anyway check out this you[tube] channel: ..."
1,Hey guys check out my new channel and our firs...
2,just for test I have to say murdev.com
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿
4,watch?v=vtaRGgvGtWQ Check this out .﻿
...,...
345,How can this have 2 billion views when there's...
346,I don't now why I'm watching this in 2014﻿
347,subscribe to me for call of duty vids and give...
348,hi guys please my android photo editor downloa...


In [24]:
#unir en un solo dataframe las caracteristicas y la columna a predecir
df = pd.concat([features, outputs], axis=1)
print(df)

                                               CONTENT  CLASS
0    huh anyway check out this youtube channel koby...      1
1    hey guys check out my new channel and our firs...      1
2                just for test i have to say murdevcom      1
3        me shaking my sexy ass on my channel enjoy _       1
4                  watchvvtarggvgtwq   check this out       1
..                                                 ...    ...
345  how can this have 2 billion views when theres ...      0
346            i dont now why im watching this in 2014      0
347  subscribe to me for call of duty vids and give...      1
348  hi guys please my android photo editor downloa...      1
349  the first billion viewed this because they tho...      0

[350 rows x 2 columns]


In [16]:
outputs

0      1
1      1
2      1
3      1
4      1
      ..
345    0
346    0
347    1
348    1
349    0
Name: CLASS, Length: 350, dtype: uint8

In [17]:
import time
import numpy as np

class NaiveBayes:

    def __init__(self):
        self.classes = None
        self.class_priors = None
        self.mean = None
        self.var = None

    def fit(self, X, y):
        start_time = time.time()
        self.classes = np.unique(y)
        self.class_priors = np.zeros(len(self.classes))
        self.mean = np.zeros((len(self.classes), X.shape[1]))
        self.var = np.zeros((len(self.classes), X.shape[1]))

        # Calcular priori de clase
        for i, c in enumerate(self.classes):
            self.class_priors[i] = np.sum(y == c) / len(y)

        # Calcular media y varianza de cada clase
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[i] = X_c.mean(axis=0)
            self.var[i] = X_c.var(axis=0)

        elapsed_time = time.time() - start_time
        print(f"Training completed in {elapsed_time:.2f} seconds.")

    def predict(self, X):
        start_time = time.time()
        predictions = []
        for x in X:
            class_probs = []
            for i, c in enumerate(self.classes):
                prior = np.log(self.class_priors[i])
                posterior = np.sum(np.log(self.normal_pdf(x, self.mean[i], self.var[i])))
                class_prob = prior + posterior
                class_probs.append(class_prob)
            predictions.append(self.classes[np.argmax(class_probs)])

        elapsed_time = time.time() - start_time
        print(f"Prediction completed in {elapsed_time:.2f} seconds.")
        return np.array(predictions)

    def normal_pdf(self, x, mean, var):
        eps = 1e-4
        numerator = np.exp(-0.5 * (x - mean)**2 / (var + eps))
        denominator = np.sqrt(2 * np.pi * var + eps)
        return numerator / denominator



Prueba de Naive Bayes con el conjunto Iris

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

# Cargar datos de ejemplo
data = load_iris()
X, y = data.data, data.target

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

# Ajustar modelo Naive Bayes
model = NaiveBayes()
model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular la puntuación en el conjunto de prueba
score = accuracy_score(y_test, y_pred)

# Imprimir precisión
print(f"Accuracy: {score:.4f}")

Training completed in 0.00 seconds.
Prediction completed in 0.00 seconds.
Accuracy: 0.9667


Ejercicio utilizando __nltk__  

In [19]:
import pandas as pd
import numpy as np
import re
import nltk
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [26]:
features['CONTENT'] = features['CONTENT'].apply(lambda x: re.sub(r'http\S+', '', x)) # Elimina enlaces
features['CONTENT'] = features['CONTENT'].apply(lambda x: re.sub(r'[^\w\s]', '', x)) # Elimina caracteres no deseados
features['CONTENT'] = features['CONTENT'].apply(lambda x: x.lower()) # Convierte a minúsculas
#nltk.download('stopwords')

In [27]:
stopwords = nltk.corpus.stopwords.words('english')
features['CONTENT'] = features['CONTENT'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)])) # Elimina stopwords
#nltk.download('wordnet')
lemmatizer = nltk.WordNetLemmatizer()
features['CONTENT'] = features['CONTENT'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()])) # Lematiza


In [30]:
corpus = [comment.split() for comment in features['CONTENT']]
model = Word2Vec(corpus,  window=5, min_count=1, workers=4)

In [32]:
# Convierte los comentarios en vectores numéricos usando Word2Vec
X = np.array([np.mean([model.wv[word] for word in comment.split() if word in model.wv], axis=0) for comment in features['CONTENT']])


/opt/homebrew/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (350,) + inhomogeneous part.

## Asignación 3

Realice el preprocesamiento que considere adecuado para que el dataset pueda ser procesado por un modelo de clasificación. __No olvide utilizar particionar en entrenamiento y validación, y realizar el preprocesamiento de manera adecuada__.

## Asignación 4

Realice una visualización de los datos en dos dimensiones. Posteriormente entrene un modelo de KNN (utilizando la clase implementada anteriormente) y valide que su modelo generaliza bien.

## Asignación 5

Entrene un modelo de Naive Bayes (utilizando la clase implementada anteriormente) y valide que su modelo generaliza bien.